In [1]:
file_location="/FileStore/tables/customer_orders-c119f.csv"
file_type = "csv"
infer_schema = "true"
first_row_is_header = "false"
delimiter = ","

# Using Spark Dataframes

In [3]:
cust_data= spark.read.load(file_location,file_type,inferSchema=infer_schema,sep=delimiter,header=first_row_is_header)

In [4]:
cust_data.printSchema()

root
-- _c0: integer (nullable = true)
-- _c1: integer (nullable = true)
-- _c2: double (nullable = true)

In [5]:
cust_data=(cust_data.withColumnRenamed('_c0','CustID')
           .withColumnRenamed('_c1','ItemID')
           .withColumnRenamed('_c2','Amount'))

In [6]:
cust_data.createOrReplaceTempView('CSData')

In [7]:
customer_spend= spark.sql(" select CustID,sum(Amount) as ToTalSpend from CSData group by CustID order by CustID "
)

In [8]:
customer_spend.show(n=20)

+------+------------------+
CustID| ToTalSpend|
+------+------------------+
 0| 5524.949999999998|
 1| 4958.600000000001|
 2| 5994.59|
 3| 4659.63|
 4| 4815.050000000002|
 5| 4561.069999999999|
 6| 5397.879999999998|
 7| 4755.070000000001|
 8| 5517.240000000001|
 9| 5322.649999999999|
 10| 4819.700000000001|
 11| 5152.290000000002|
 12| 4664.589999999998|
 13| 4367.62|
 14| 4735.030000000001|
 15| 5413.510000000001|
 16| 4979.06|
 17| 5032.679999999999|
 18| 4921.27|
 19|5059.4299999999985|
+------+------------------+
only showing top 20 rows

# Using Spark RDD

In [10]:
txt=spark.sparkContext.textFile(file_location)


In [11]:
def line(l):
  row=l.split(',')
  return (int(row[0]),round(float(row[2]),2))

In [12]:
rows=txt.map(line)

In [13]:
total=rows.reduceByKey(lambda x,y:x+y)

In [14]:
print(sorted(total.collect()))

[(0, 5524.949999999999), (1, 4958.600000000001), (2, 5994.59), (3, 4659.63), (4, 4815.050000000001), (5, 4561.07), (6, 5397.879999999999), (7, 4755.069999999999), (8, 5517.24), (9, 5322.65), (10, 4819.7), (11, 5152.289999999999), (12, 4664.589999999999), (13, 4367.619999999999), (14, 4735.030000000001), (15, 5413.51), (16, 4979.06), (17, 5032.68), (18, 4921.269999999999), (19, 5059.43), (20, 4836.860000000001), (21, 4707.41), (22, 5019.449999999999), (23, 4042.65), (24, 5259.92), (25, 5057.610000000001), (26, 5250.4), (27, 4915.89), (28, 5000.709999999999), (29, 5032.530000000001), (30, 4990.72), (31, 4765.049999999999), (32, 5496.05), (33, 5254.660000000002), (34, 5330.8), (35, 5155.420000000001), (36, 4278.049999999999), (37, 4735.200000000001), (38, 4898.460000000001), (39, 6193.110000000001), (40, 5186.429999999999), (41, 5637.620000000001), (42, 5696.840000000002), (43, 5368.83), (44, 4756.890000000001), (45, 3309.3799999999997), (46, 5963.109999999999), (47, 4316.299999999998), (48, 4384.33), (49, 4394.6), (50, 4517.2699999999995), (51, 4975.220000000001), (52, 5245.0599999999995), (53, 4945.300000000001), (54, 6065.390000000001), (55, 5298.089999999999), (56, 4701.02), (57, 4628.4), (58, 5437.73), (59, 5642.889999999999), (60, 5040.709999999999), (61, 5497.4800000000005), (62, 5253.3200000000015), (63, 5415.1500000000015), (64, 5288.689999999999), (65, 5140.35), (66, 4681.92), (67, 4505.79), (68, 6375.450000000001), (69, 5123.01), (70, 5368.249999999999), (71, 5995.660000000002), (72, 5337.439999999999), (73, 6206.200000000001), (74, 4647.130000000001), (75, 4178.5), (76, 4904.210000000001), (77, 4327.73), (78, 4524.51), (79, 3790.5699999999997), (80, 4727.86), (81, 5112.709999999999), (82, 4812.49), (83, 4635.8), (84, 4652.9400000000005), (85, 5503.43), (86, 4908.810000000001), (87, 5206.4), (88, 4830.55), (89, 4851.48), (90, 5290.41), (91, 4642.26), (92, 5379.279999999999), (93, 5265.75), (94, 4475.570000000001), (95, 4876.839999999998), (96, 3924.2300000000005), (97, 5977.1900000000005), (98, 4297.259999999999), (99, 4172.29)]